In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/crop-production-in-india/crop_production.csv


In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import joblib


In [3]:

df = pd.read_csv("../input/crop-production-in-india/crop_production.csv")
print("Shape:", df.shape)
df.head()


Shape: (246091, 7)


,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [4]:

df.dropna(inplace=True)
df = df[(df['Area'] > 0) & (df['Production'] > 0)]
df['Yield'] = df['Production'] / df['Area']
df = df[df['Yield'] > 0.01]
threshold = df['Yield'].quantile(0.99)
df = df[df['Yield'] < threshold]
df.reset_index(drop=True, inplace=True)

df.describe()


,Crop_Year,Area,Production,Yield
count,236211.000000,2.362110e+05,2.362110e+05,236211.000000
mean,2005.676137,1.230315e+04,3.915992e+04,3.944465
std,4.975792,5.135030e+04,3.048416e+05,9.623429
min,1997.000000,1.000000e-01,2.000000e-02,0.010101
25%,2002.000000,9.000000e+01,9.700000e+01,0.529201
50%,2006.000000,6.250000e+02,7.520000e+02,1.000000
75%,2010.000000,4.610000e+03,6.868500e+03,2.315691
max,2015.000000,8.580100e+06,1.775796e+07,83.594595


In [5]:
from sklearn.preprocessing import LabelEncoder

le_crop = LabelEncoder()
le_state = LabelEncoder()
le_season = LabelEncoder()

df['Crop'] = le_crop.fit_transform(df['Crop'])
df['State_Name'] = le_state.fit_transform(df['State_Name'])
df['Season'] = le_season.fit_transform(df['Season'])




In [6]:
X = df[['Crop', 'State_Name', 'Season', 'Area']]
y = df['Yield']


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)
rmse_lr = mean_squared_error(y_test, y_pred_lr, squared=False)
print(f"Linear Regression RMSE: {rmse_lr:.2f} kg/ha")


Linear Regression RMSE: 9.17 kg/ha


In [9]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)
rmse_dt = mean_squared_error(y_test, y_pred_dt, squared=False)
print(f"Decision Tree RMSE: {rmse_dt:.2f} kg/ha")


Decision Tree RMSE: 3.49 kg/ha


In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
print(f"Random Forest RMSE: {rmse_rf:.2f} kg/ha")


Random Forest RMSE: 3.01 kg/ha


In [11]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
print(f"XGBoost RMSE: {rmse_xgb:.2f} kg/ha")


XGBoost RMSE: 2.79 kg/ha


In [12]:

models = {
    "Linear Regression": rmse_lr,
    "Decision Tree": rmse_dt,
    "Random Forest": rmse_rf,
    "XGBoost": rmse_xgb
    # "SVR": rmse_svr,       # <- Commented out
    # "KNN": rmse_knn        # <- Commented out
}

# Sort by RMSE score
sorted_models = dict(sorted(models.items(), key=lambda item: item[1]))

print("🔍 Model RMSE Comparison:")
for model, score in sorted_models.items():
    print(f"{model:20s} : {score:.2f} kg/ha")


🔍 Model RMSE Comparison:
XGBoost              : 2.79 kg/ha
Random Forest        : 3.01 kg/ha
Decision Tree        : 3.49 kg/ha
Linear Regression    : 9.17 kg/ha


In [13]:
!pip install joblib


In [14]:
import pandas as pd
import joblib



In [15]:
xgb = joblib.load('xgb_yield_model.joblib')



FileNotFoundError: [Errno 2] No such file or directory: 'xgb_yield_model.joblib'

In [ ]:
print(X_test.columns)



In [ ]:

sample = X_test.iloc[[0]]  # Double brackets keep it a DataFrame
print(sample)


In [ ]:

predicted_yield = xgb.predict(sample)[0]
print(f"Predicted Yield: {predicted_yield:.2f} kg/hectare")


In [ ]:
for i in range(5):
    sample = X_test.iloc[[i]]
    pred = xgb.predict(sample)[0]
    actual = y_test.iloc[i]
    print(f"Predicted: {pred:.2f} | Actual: {actual:.2f}")


In [ ]:
import matplotlib.pyplot as plt

preds = xgb.predict(X_test)
plt.figure(figsize=(8,5))
plt.scatter(y_test, preds, alpha=0.6)
plt.xlabel("Actual Yield (kg/ha)")
plt.ylabel("Predicted Yield (kg/ha)")
plt.title("Predicted vs Actual Yield")
plt.plot([0, max(y_test)], [0, max(y_test)], color='red')  # ideal line
plt.grid(True)
plt.show()
